# Plotting of  Aluminosilicate Phase Diagram from Berman(1988) database of pure solid phases (standard state properties)
## Data are adjusted datapoints from Holdaway1993
## Concepts Demonstrated
* Loading a thermodynamic database
* Imposing experimental priors

## Import libraries ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook
import matplotlib as mpl
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy import stats
import scipy as sp
from scipy.optimize import least_squares

import os
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/msghiorso/ENKI/PhaseObjC')

import thermoengine
from thermoengine import phases
from thermoengine import model
from thermoengine import calibrate

import importlib


In [ ]:
prior = pd.read_excel('phase-rxn-data/priors-V0.xlsx')
mask_Ky = prior['phase']=='Ky'
mask_And = prior['phase']=='And'
# mask_Sil = (prior['phase']=='Sil') & (prior['V']<4.99) & (prior['V']>4.98)
mask_Sil = (prior['phase']=='Sil')

N_Ky = np.sum(mask_Ky)
N_And = np.sum(mask_And)
N_Sil = np.sum(mask_Sil)

In [ ]:
def lin_fun(x, t, y):
    return x[1] + x[0]*t - y

kde_And = stats.gaussian_kde(prior['V'][mask_And])
x_And = np.linspace(5.14,5.16,1001)
y_And = kde_And.evaluate(x_And)
cdf_And = np.array([kde_And.integrate_box_1d(-np.inf,ix) for ix in x_And])



kde_Ky = stats.gaussian_kde(prior['V'][mask_Ky])
x_Ky = np.linspace(4.36,4.46,1001)
y_Ky = kde_Ky.evaluate(x_Ky)
cdf_Ky = np.array([kde_Ky.integrate_box_1d(-np.inf,ix) for ix in x_Ky])

kde_Sil = stats.gaussian_kde(prior['V'][mask_Sil])
x_Sil = np.linspace(4.95,5.01,1001)
y_Sil = kde_Sil.evaluate(x_Sil)
cdf_Sil = np.array([kde_Sil.integrate_box_1d(-np.inf,ix) for ix in x_Sil])


In [ ]:
cdf_bnd_And = np.interp([.25,.75], cdf_And, x_And)
avg_And, err_And = np.mean(cdf_bnd_And), np.diff(cdf_bnd_And)[0]/1.35

cdf_bnd_Ky = np.interp([.25,.75], cdf_Ky, x_Ky)
avg_Ky, err_Ky = np.mean(cdf_bnd_Ky), np.diff(cdf_bnd_Ky)[0]/1.35

cdf_bnd_Sil = np.interp([.25,.75], cdf_Sil, x_Sil)
avg_Sil, err_Sil = np.mean(cdf_bnd_Sil), np.diff(cdf_bnd_Sil)[0]/1.35



ynorm_Ky = sp.stats.norm.pdf(x_Ky, loc=avg_Ky, scale=err_Ky)
ynorm_Ky *= .3/np.max(ynorm_Ky)

ynorm_And = sp.stats.norm.pdf(x_And, loc=avg_And, scale=err_And)
ynorm_And *= .3/np.max(ynorm_And)

ynorm_Sil = sp.stats.norm.pdf(x_Sil, loc=avg_Sil, scale=err_Sil)
ynorm_Sil *= .3/np.max(ynorm_Sil)

In [ ]:
print('Ky = ', np.round(avg_Ky, decimals=4), ' +/- ', np.round(err_Ky, decimals=4))
print('And = ', np.round(avg_And, decimals=4), ' +/- ', np.round(err_And, decimals=4))
print('Sil = ', np.round(avg_Sil, decimals=4), ' +/- ', np.round(err_Sil, decimals=4))

print('=====')
print('Ky = ', np.round(4.408-avg_Ky, decimals=4))
print('And = ', np.round(5.148-avg_And, decimals=4))
print('Sil = ', np.round(4.986-avg_Sil, decimals=4))


* Fe2O3 correction is negligible compared to spread in data
* These provide excellent priors consistent with Holdaway1993, but with greater uncertainty

In [ ]:

plt.figure()
poly_Ky = np.polyfit(prior['Fe2O3'][mask_Ky], prior['V'][mask_Ky], 1)
res_Ky = least_squares(lin_fun, poly_Ky, loss='soft_l1',
              args=(prior['Fe2O3'][mask_Ky], prior['V'][mask_Ky]))

plt.plot(prior['Fe2O3'][mask_Ky], prior['V'][mask_Ky],'ko')
plt.plot(prior['Fe2O3'][mask_Ky], prior['V_corr'][mask_Ky],'rx')
plt.plot([0,.5],np.polyval(res_Ky.x,[0,.5]),'r--')
plt.plot(.3*y_Ky/np.max(y_Ky), x_Ky, 'b-')
plt.plot(ynorm_Ky, x_Ky, 'b:')
plt.xlim(0, .5)
plt.title('Ky')


plt.figure()
poly_And = np.polyfit(prior['Fe2O3'][mask_And], prior['V'][mask_And], 1)
res_And = least_squares(lin_fun, poly_And, loss='soft_l1',
              args=(prior['Fe2O3'][mask_And], prior['V'][mask_And]))
plt.plot(prior['Fe2O3'][mask_And], prior['V'][mask_And],'ko')
plt.plot(prior['Fe2O3'][mask_And], prior['V_corr'][mask_And],'rx')
plt.plot([0,.5],np.polyval(res_And.x, [0,.5]), 'r--')
plt.plot(.3*y_And/np.max(y_And), x_And, 'b-')
plt.plot(ynorm_And, x_And, 'b:')
plt.xlim(0, .5)
plt.title('And')

plt.figure()
poly_Sil = np.polyfit(prior['Fe2O3'][mask_Sil], prior['V'][mask_Sil], 1)
res_Sil = least_squares(lin_fun, [0,4.985],  f_scale=.002,
              args=(prior['Fe2O3'][mask_Sil], prior['V'][mask_Sil]))
plt.plot(prior['Fe2O3'][mask_Sil], prior['V'][mask_Sil],'ko')
plt.plot(prior['Fe2O3'][mask_Sil], prior['V_corr'][mask_Sil],'rx')
plt.plot([0,.5],np.polyval(res_Sil.x, [0,.5]), 'r--')
plt.plot(.3*y_Sil/np.max(y_Sil), x_Sil, 'b-')
plt.plot(ynorm_Sil, x_Sil, 'b:')
# plt.xlim(0, .5)
plt.title('Sil')




In [ ]:
plt.figure()
plt.plot(x_Sil, cdf_Sil, 'r-')
plt.title('Sil')

plt.figure()
plt.plot(x_Ky, cdf_Ky, 'r-')
plt.title('Ky')

plt.figure()
plt.plot(x_And, cdf_And, 'r-')
plt.title('And')


In [ ]:
avg_And

In [ ]:
sp.stats.norm.cdf([-.675, .675])

In [ ]:
.675*2